In [1]:
import numpy as np
import os
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Users\amanda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\amanda\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\amanda\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\amanda\AppData\Roaming\P

In [11]:
def sample(preds, temp=1.0):
    '''
    Helper function for sampling from softmax with different temperatures.
    
    Inputs:
    preds: output of softmax function
    temp: temperature to scale by
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [2]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()
    
# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers as well as reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [3]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)

In [5]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [43]:
# define the LSTM model 
# (single layer with 200 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))

In [58]:
# load the network weights from best checkpoint
#filename = "weights-improvement-20-1.7396.hdf5"
filename = "weights-improvement-61-1.1807.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [59]:
# set seed for emissions
seed = "shall i compare thee to a summer's day?\n"
seed_to_int = [char_to_int[char] for char in seed]

# generate emissions
pattern = seed_to_int

temps = [0.01, 0.1, 0.25, 0.75, 1.5]    
for temp in temps:    
    print('Temperature', temp)
    
    # generate characters
    newlines = 0
    while newlines < 14:
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)[0]

        # sample according to temperature
        idx = sample(prediction, temp)

        result = int_to_char[idx]
        pattern.append(idx)
        pattern = pattern[1:len(pattern)]
        
        if result == '\n':
            newlines += 1
        
        # output result 
        sys.stdout.write(result)
        sys.stdout.flush()
    print()

Temperature 0.01
thou art thy self the thorgh of all his rriae
helple then

C:\Users\amanda\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


 io ore oo self of mane and sens,
  aut hov the paintec boln to bod,
ms vhr derior th the datth of you i dend seee for where,
the basest cudrtt of the tomen of yourh,
when in the beyt in of his jovertinn bace,
where ale iy krve to be a derer (hrcoug watte:
the cout of thee may eete more to shee,
  thie mf hia cbr tomk my hode torevs or tey,
with that my silful wall my self and tuine,
  and titu an to teif the vorld ofr move ther boln,
that the then then the world shou dars sooved,
and to the temner of hiart say hy dedd,
the bester asb oot the faythen of the dey,

Temperature 0.1
and oayet of made the dan that teee thou are,
the houbrt with a cuend to ces'd thint to bren.
to sey i fane toeer the farrh sfront,
that in the breath the caytht mogt thou frow,
  and yith all the wornd will wou,s soopowed tore,
and that in thee what fear fanh fan to she hrase,
whocst if wou kavi the simed oitehnged,
and that in thee whth pthcrs what they see?
thou dost love hove the sheek ofrer iash,
where bes